Importing necessary packages!

In [1]:
import spacy
import json
import random

A function for loading the data from JSON files!

In [2]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

A function for exporting to a JSON file!

In [3]:
def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

A function for **training** our custom NER NLP model!


In [4]:
def train_ner_nlp_model(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en") #A new blank ENGLISH NLP object.
    #Checking if a NER component already exists in the pipeline.
    if("ner" not in nlp.pipe_names): 
        #Add new NER pipe or else get the existing ner pipe.
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    #Creating a list of other pipes, except the NER pipe.
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    #For training our specific pipe, we need to disable other pipes.
    with nlp.select_pipes(disable=other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print (losses)
        return (nlp)